In [ ]:
"""
Created on Tue May  4 00:11:06 2021

@author: YUSUF BARAN TANRIVERDİ


"""
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# accuracy_score average_precision_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, recall_score, roc_auc_score, balanced_accuracy_score, precision_score
from sklearn.metrics import roc_curve

data = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')


In [ ]:
# Turn categorical values into label encoding
# ref: https://pbpython.com/categorical-encoding.html

def label_encoding(data, feature_name):
    # encodes with an alphabetical order  of original titles
    data[feature_name] = data[feature_name].astype('category')
    data[feature_name] = data[feature_name].cat.codes
    return


label_encoding(data, 'gender')
label_encoding(data, 'ever_married')
label_encoding(data, 'work_type')
label_encoding(data, 'Residence_type')
label_encoding(data, 'smoking_status')

In [ ]:
data

SPECIAL CASES
1) NaN values
A very common way to replace missing values is using a median.
Replace using median 

ref: https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b

2) UNKNOWN values of smoking status
 What to do? 
 They shouldn't play as a factor category- 
 i.e they should be neutral on the subject of smoking is whether harmful or not!
 For now, let's pass them as another category.
 

In [ ]:
median = data['bmi'].median()
data['bmi'].fillna(median, inplace=True)

In [ ]:
sns.pairplot(data.iloc[:, 5:], hue='stroke') 
plt.show()

It is seen that data is imbalanced heavily. Let's experiment with that before handling.

**Create Datasets**

In [ ]:
# create datasets    
xy = data.iloc[:, 1:].to_numpy()
np.random.shuffle(xy)


x = xy[:, :-1]
y = xy[:, -1]

**Some Useful Functions**

In [ ]:
def convert_one_hot_vectors(y):
    target_list = []
    one_hot = np.eye(2)
    for target in y:
        if target == 0:
            target_list.append(one_hot[0])
        if target == 1:
            target_list.append(one_hot[1])
    y = np.asarray(target_list)
    
    #x, y = shuffle_data(x, y)
    return y

def get_accuracy(y_preds, y_labels):
    matches = 0
    for boolean in y_preds == y_labels:
        if boolean == True: matches = matches +1
    return matches/ len(y_labels)* 100

#ref:https://scikit-learn.org/stable/modules/model_evaluation.html
def confusion_matrix_scorer(y_preds, y_labels):
    cm = confusion_matrix(y_labels, y_preds)
    return {'tn': cm[0, 0], 'fp': cm[0, 1],
            'fn': cm[1, 0], 'tp': cm[1, 1]}

#ref: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
def plot_roc_auc_curve(y_preds, y_labels, model_name):
    ns_auc = roc_auc_score(y_labels, np.zeros(y_labels.shape))
    lr_auc = roc_auc_score(y_labels, y_preds)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print(f'{model_name}: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y, np.zeros(y.shape))
    lr_fpr, lr_tpr, _ = roc_curve(y, preds)
    # plot the roc curve for the model
    ns_curve, = plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label=f'{model_name}')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()
    ns_curve.remove()

def print_metrics(y_preds, y_labels, model_name):
    print("\n MODEL: ", model_name )
    
    print("Accuracy:", get_accuracy(y_preds, y_labels))
    print("Null rate of test:", get_accuracy(y_preds, np.zeros(y.shape)))
    print("\n CONFUSION MATRIX")
    print(confusion_matrix_scorer(y_preds, y_labels))

    # recall is tp/ (tp+fn)
    print("Recall: ", recall_score(y_preds, y_labels))
    
    # precision is tp/(tp+fp)
    print("Precision:", precision_score(y_preds, y_labels))
    
    # (recall+ specificity)/2
    print("Balanced Accuracy:", balanced_accuracy_score(y_labels, y_preds)* 100)


def fit_LinearRegression():
    regressor = LinearRegression()
    tmp_y = convert_one_hot_vectors(y)
    regressor.fit(x,tmp_y)
        
    probs = regressor.predict(x)
    return  probs, regressor

EXP1 LINEAR REGRESSION

In [ ]:
probs, _  = fit_LinearRegression()
preds = np.argmax(probs, axis= 1)
print_metrics(preds, y, 'Linear Regression (imbalanced data)')
# balanced accuracy in this case would be 0.5


print("PREVELANCE -> pr(stroke|sample) = ",len(data[data['stroke'] == 1])/len(data))
# see the scatter plot also.




It always give 'negative'. Imbalanced target problem?   Yes, solutions might be:
    
SMOTE

MODEL CHANGING

DIFFERENT METRIC EVALUATION 

(ref: https://towardsdatascience.com/regression-for-imbalanced-data-with-application-edf93517247c)

EXP2 SMOTE - REGRESSION

In [ ]:
from imblearn.over_sampling import SMOTENC

#CALL RAW DATA
x = xy[:, :-1]
y = xy[:, -1]

#SMOTE the data with SMOTENC algorithm
cat_col_index = [0, 4, 5, 6, 9]
oversample = SMOTENC(categorical_features=cat_col_index, k_neighbors=2)
x, y = oversample.fit_resample(x, y)

df = pd.DataFrame(x, columns= ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married','work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'])

df['stroke'] = y
# print(df)
#sns.pairplot(df.iloc[:, 5:], hue='stroke') 
#plt.show()
#sns.pairplot(df.iloc[:, 5:], hue='stroke') 
#plt.show()
print(" \n AFTER SMOTENC...")

probs, linearModelv2 = fit_LinearRegression()
preds = np.argmax(probs, axis= 1)

print_metrics(preds, y, 'Linear Regression after SMOTE')

print(" NEW PREVELANCE ->pr(stroke|sample) = ", len(df[df['stroke'] == 1])/len(df))

In [ ]:
sns.pairplot(df.iloc[:, 5:], hue='stroke') 
plt.show()

EXP3 DEEP LEARNING METHODS

REFERENCES:
 https://towardsdatascience.com/handling-overfitting-in-deep-learning-models-c760ee047c6e

 https://towardsdatascience.com/dont-overfit-how-to-prevent-overfitting-in-your-deep-learning-models-63274e552323

 https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809


In [ ]:
# import libs and define some useful functions
from keras.layers import *
import tensorflow as tf
import keras

def get_evaluation_accuracy(history):
    return 100 * history['accuracy']


def plot_metrics(model_name, history, metric_name):
    plt.figure()
    e = range(1, len(history.history[metric_name]) + 1)
    plt.plot(e, history.history[metric_name], 'bo', label=metric_name)
    plt.plot(e, history.history[f'val_{metric_name}'], 'b', label=f'val_{metric_name}')
    plt.xlabel('Epoch')
    plt.ylabel(f'{metric_name}')
    plt.legend(loc='lower right')
    plt.title(f'Comparing training and validation loss of {model_name}')
    plt.savefig(f'./{model_name}.png')

def metrics_table_for_evaluation(model_name, history):
    tp = history['true_pos']
    fp = history['false_pos']
    tn = history['true_neg']
    fn = history['false_neg']
    
    cm = pd.DataFrame(index=('predicted YES', 'predicted NO'), columns=('actual YES', 'actual NO'))
    cm.iloc[0,0] = np.mean(tp)
    cm.iloc[0,1] = np.mean(fn)
    cm.iloc[1,0] = np.mean(fp)
    cm.iloc[1,1] = np.mean(tn)
    
    mcs = pd.DataFrame(index=[model_name], columns=('recall (TPR)', 'specificty', 'sensitivity', 'precision', 'false positive rate', 'accuracy'))
    
    mcs.iloc[0,0] = np.mean(tp)/ (np.mean(tp)+ np.mean(fp))
    mcs.iloc[0,1] = np.mean(tn)/ (np.mean(tn)+ np.mean(fp)) 
    mcs.iloc[0,2] = np.mean(tp)/ (np.mean(tp)+ np.mean(fn)) 
    mcs.iloc[0,3] = np.mean(tp)/ (np.mean(tp)+ np.mean(fp)) 
    mcs.iloc[0,4] = np.mean(fp)/ (np.mean(tn)+ np.mean(fp)) 
    mcs.iloc[0,5] = np.mean(history['accuracy']) 
    
    print(cm)
    print(mcs)

In [ ]:
#HYPERPARAMATERS

LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
EPOCHS= 100
BATCH_SIZE = 10
LOSS_FN = 'binary_crossentropy'

In [ ]:
# create train_dataset
# test split %90

boundary = int(len(x) *0.1)
x_train = x[boundary:]
y_train = y[boundary:]
    
# create test_dataset
x_test = x[:boundary]
y_test = y[:boundary]

DEFINE MODEL with a dropout layer.

In [ ]:
def dropout_dnn(number_of_classes=2, lr=LEARNING_RATE, dr=DROPOUT_RATE):
    model = keras.Sequential([
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dropout(dr),
        Dense(64, activation='relu'),
        Dropout(dr),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss=LOSS_FN,
                  metrics=['accuracy', 
                           tf.keras.metrics.TrueNegatives(name='true_neg'),
                           tf.keras.metrics.FalsePositives(name='false_pos'),
                           tf.keras.metrics.FalseNegatives(name='false_neg'),
                           tf.keras.metrics.TruePositives(name='true_pos'),
                           ],
                  optimizer=tf.keras.optimizers.Adam(lr))
    return model



To stop training when no change happens:

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

**Tranining**

In [ ]:
densenet = dropout_dnn()
dropout_history = densenet.fit(x_train, y_train, validation_split=0.2, 
                               batch_size=BATCH_SIZE, verbose=1, epochs=EPOCHS
                               )

#save model
tf.keras.models.save_model(
    densenet,
    './models/densenet.h5'
)


pd.DataFrame.from_dict(dropout_history.history).to_csv('./dropout_histor.csv', index=False)



In [ ]:
plot_metrics(history=dropout_history, metric_name='loss', model_name='densenet')
plot_metrics(history=dropout_history, metric_name='accuracy', model_name='densenet')

In [ ]:
eval_history = densenet.evaluate(x_test, y_test, return_dict=True)

metrics_table_for_evaluation('MLP test', eval_history)

In [ ]:
metrics_table_for_evaluation('MLP train', dropout_history.history)




In [ ]:
print("NULL RATE")
densenet.evaluate(x_test, np.zeros(y_test.shape))

print("FULL RATE")
indices = [i for i in range(len(y)) if y[i]==1]
densenet.evaluate(x_test, np.ones(y_test.shape))

EXP4 XGBOOST

In [ ]:
from collections import Counter

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [ ]:
# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)


In [ ]:
model.fit(x, y)

In [ ]:
evals_result = model.score(x_test, y_test)

evals_result